In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install transformers

In [3]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2024-06-02 09:32:11--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   174MB/s    in 0.2s    

2024-06-02 09:32:11 (174 MB/s) - 'VnCoreNLP-1.1.1.jar' saved [27412575/27412575]

--2024-06-02 09:32:12--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [4]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2024-06-02 09:32:17--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 108.159.227.26, 108.159.227.61, 108.159.227.65, ...
Connecting to public.vinai.io (public.vinai.io)|108.159.227.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: 'PhoBERT_base_transformers.tar.gz.1'

PhoBERT_base_transf 100%[===================>] 307.47M   256MB/s    in 1.2s    

2024-06-02 09:32:19 (256 MB/s) - 'PhoBERT_base_transformers.tar.gz.1' saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [5]:
!pip install vncorenlp


In [6]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/kaggle/working/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

text = "Đại học Bách Khoa Hà Nội."

word_segmented_text = rdrsegmenter.tokenize(text)
print(word_segmented_text)

[['Đại_học', 'Bách_Khoa', 'Hà_Nội', '.']]


In [7]:
import re
import pandas as pd

test_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/test.csv'
test_set = pd.read_csv(test_path)
test_set['Sentence'] = test_set['Sentence'].apply(rdrsegmenter.tokenize).apply(lambda x: ' '.join(x[0]))

df = test_set.copy()
df

,Sentence,FACILITY,LECTURER,OTHERS,TRAINING_PROGRAM
0,nói tiếng anh lưu_loát,0,1,0,0
1,giáo_viên rất vui_tính,0,1,0,0
2,cô max có_tâm,0,1,0,0
3,giảng bài thu_hút dí_dỏm,0,1,0,0
4,giáo_viên không giảng_dạy kiến_thức hướng_dẫn ...,0,2,0,0
...,...,...,...,...,...
3161,các slide khó hiểu ngôn_ngữ trong slide phức_t...,0,2,0,0
3162,giáo_viên giảng_dạy có tâm_huyết,0,1,0,0
3163,chia_sẻ cho em nhiều điều hay,0,1,0,0
3164,em tiếp_thu chậm,0,2,0,0


In [43]:
3166*4

12664

In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Sample data for illustration
# data = train_set[['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']].copy()
data = df.copy()

# Create DataFrame
df = pd.DataFrame(data)

# Select specific columns
selected_columns = ['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']
data_to_encode = df[selected_columns]

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
encoded_data = encoder.fit_transform(data_to_encode)

# Convert the result to a DataFrame for better readability
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(selected_columns))

# If you want to list the values
encoded_values_list = encoded_df.values
# print(encoded_values_list)

In [9]:
encoded_values_list.shape

(3166, 16)

In [11]:
import os
import random
import json
import torch
import torch.nn as nn
from functools import partial
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from huggingface_hub import HfApi

# import evaluate
import numpy as np
# from datasets import load_dataset
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, TrainingArguments

2024-06-02 09:40:26.279220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 09:40:26.279284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 09:40:26.281031: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
import re
import pandas as pd

# train_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/train.csv'
# test_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/test.csv'

# train_set = pd.read_csv(train_path)
# test_set = pd.read_csv(test_path)

test_id, test_text, test_labels = [], [], []

test_id = [('test_' + str(i)) for i in range(df.shape[0])]
test_text = list(df['Sentence'].values)
test_labels = encoded_values_list
test_labels = torch.tensor(test_labels)


In [13]:
import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "vinai/phobert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Example input data
texts = test_text.copy()

# Tokenize input data
inputs = tokenizer(
    texts,
    max_length = 256, 
    padding='max_length',
    truncation=True,
    return_tensors="pt"  # Return PyTorch tensors
)
    

# Print the tokenized inputs
print(inputs)

{'input_ids': tensor([[    0,    96,   355,  ...,     1,     1,     1],
        [    0,   960,    59,  ...,     1,     1,     1],
        [    0,   106, 23371,  ...,     1,     1,     1],
        ...,
        [    0,   288,    13,  ...,     1,     1,     1],
        [    0,   193,  4747,  ...,     1,     1,     1],
        [    0,   193,    10,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [38]:
inputs['input_ids'].shape

torch.Size([3166, 256])

In [14]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "/kaggle/working/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 16, output_hidden_states=False,
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


In [15]:
# Specify a path
PATH = '/kaggle/input/absa-phobert-sehilnlf/state_dict_model_10.pt'

In [16]:
# Load
model = RobertaForSequenceClassification.from_pretrained(
    "/kaggle/working/PhoBERT_base_transformers/model.bin",
    config=config
)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)
model.load_state_dict(torch.load(PATH))
model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

## EVALUATE

In [42]:
device = 'cuda'

model.eval()
# eval_loss, eval_accuracy = 0, 0
# nb_eval_steps, nb_eval_examples = 0, 0
# eval_f1 = 0
# total_loss = 0
# total_samples = 0
# correct_samples = 0

b_input_ids, b_input_mask = inputs['input_ids'], inputs['attention_mask']
b_labels = test_labels
b_input_ids = b_input_ids.to(device)
b_input_mask = b_input_mask.to(device)
b_labels = b_labels.to(device)

logits = []
for i in range(len(b_labels)):
    with torch.no_grad():
        outputs = model(b_input_ids[i].reshape(1,-1),
                        token_type_ids=None,
                        attention_mask=b_input_mask[i].reshape(1,-1))

        logit = outputs.logits  # keep this tensor on the GPU
        logits.append(list(np.array(logit[0].cpu())))

In [39]:
len(logits)

4

In [22]:
def correct_sample(logits, b_labels):
    logits = torch.tensor(logits).reshape(-1,4,4)
    label_ids = b_labels.view(-1, 4, 4)

    # Convert one-hot encoded labels to class indices
    label_ids_indices = torch.argmax(label_ids, dim=-1).cpu()

    # Reshape logits and label_ids
    batch_size, num_capsules, num_classes = logits.shape
    logits_flat = logits.view(batch_size * num_capsules, num_classes)
    label_ids_flat = label_ids_indices.view(batch_size * num_capsules)

    # Calculate loss
    # loss = criterion(logits_flat, label_ids_flat)
    # total_loss += batch_size * loss.item()
    # total_samples += batch_size * num_classes

    pred = logits_flat.argmax(dim=1)
    correct_samples = (label_ids_flat == pred).long().sum().item()
    return correct_samples

correct_sample(logits, b_labels)

11790

In [44]:
(len(b_labels) * 4)

12664

In [24]:
def return_preds_and_labels(logits, b_labels):
    logits = torch.tensor(logits).reshape(-1,4,4)
    label_ids = b_labels.view(-1, 4, 4)

    # Convert one-hot encoded labels to class indices
    label_ids_indices = torch.argmax(label_ids, dim=-1).cpu()

    # Reshape logits and label_ids
    batch_size, num_capsules, num_classes = logits.shape
    logits_flat = logits.view(batch_size * num_capsules, num_classes)
    label_ids_flat = label_ids_indices.view(batch_size * num_capsules)

    # Calculate loss
    # loss = criterion(logits_flat, label_ids_flat)
    # total_loss += batch_size * loss.item()
    # total_samples += batch_size * num_classes
    pred = logits_flat.argmax(dim=1)
    return pred, label_ids_flat

pred, label = return_preds_and_labels(logits, b_labels)

In [36]:
label.shape

torch.Size([12664])

In [25]:
from sklearn.metrics import classification_report
print(classification_report(label, pred))
# 0: None, 1: 'Positive', 2: 'Negative', 3: 'Normal'

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      9498
           1       0.88      0.88      0.88      1590
           2       0.80      0.81      0.81      1409
           3       0.46      0.04      0.07       167

    accuracy                           0.93     12664
   macro avg       0.78      0.67      0.68     12664
weighted avg       0.93      0.93      0.93     12664



In [55]:
# topic
# 'FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM'
print(classification_report(label.reshape(-1,4).argmax(dim = 1), pred.reshape(-1,4).argmax(dim = 1)))

              precision    recall  f1-score   support

           0       0.45      0.95      0.61       145
           1       0.94      0.95      0.94      2290
           2       0.79      0.07      0.13       159
           3       0.82      0.78      0.80       572

    accuracy                           0.87      3166
   macro avg       0.75      0.69      0.62      3166
weighted avg       0.89      0.87      0.86      3166



## INFERENCE

In [26]:
def inference(texts):
    # segmenter
    word_segmented_text = rdrsegmenter.tokenize(text)

    # Tokenize input data
    inputs = tokenizer(
        texts,
        max_length = 256, 
        padding='max_length',
        truncation=True,
        return_tensors="pt"  # Return PyTorch tensors
    )

    inputs.to(DEVICE)
    
    with torch.no_grad():
        outputs = model(inputs['input_ids'],
                    token_type_ids=None,
                    attention_mask=inputs['attention_mask'])

    logits = outputs.logits
    logits = logits.view(-1, 4, 4)
    logits = torch.argmax(logits, dim=-1)[0]
    
    print(logits)
    topic = ['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']
    sentiment = ['None', 'Positive', 'Negative', 'Normal']
    for i in range(len(logits)):
        if logits[i] == 0:
            continue
        print('Topic:', topic[i])
        print('Sentiment:', sentiment[logits[i]])

        break
        
inference(["giao bài_tập quá nhiều"])

tensor([0, 2, 0, 0], device='cuda:0')
Topic: LECTURER
Sentiment: Negative


In [27]:
logits = torch.tensor([0, 2, 0, 0], device='cuda:0')
logits

tensor([0, 2, 0, 0], device='cuda:0')

In [28]:
(torch.tensor([0, 2, 0, 0], device='cuda:0') == torch.tensor([0, 2, 0, 0], device='cuda:0'))

tensor([True, True, True, True], device='cuda:0')

In [29]:
inference(["chưa giỏi chuyên_môn cho lắm"])

tensor([0, 2, 0, 0], device='cuda:0')
Topic: LECTURER
Sentiment: Negative


In [30]:
inference(["chưa áp dụng công nghệ thông tin"])

tensor([0, 2, 0, 0], device='cuda:0')
Topic: LECTURER
Sentiment: Negative


In [31]:
inference(["giáo_viên rất vui_tính"])

tensor([0, 1, 0, 0], device='cuda:0')
Topic: LECTURER
Sentiment: Positive


In [32]:
inference(["có đôi_lúc nói hơi nhanh làm sinh_viên không theo kịp"])

tensor([0, 2, 0, 0], device='cuda:0')
Topic: LECTURER
Sentiment: Negative


In [33]:
inference(["giáo trình cũ không có gì mới"])

tensor([0, 0, 0, 2], device='cuda:0')
Topic: TRAINING_PROGRAM
Sentiment: Negative
